## Importing the important libraries 

retinaface is used to detect where the face is present in the entire face and 
deep face is used to create the  embeddings 
cv2 is used to read the image from the file
os is used to navigate the directories and fetch the files
numpy is used to manipulate the necessary data structures

In [1]:
from retinaface import RetinaFace
from deepface import DeepFace
from matplotlib import pyplot as plt
import cv2
import os
import datetime as datetime
import pandas as pd
import numpy as np

FAISS is FaceBook AI similarity search and 

In [2]:
import faiss
from datetime import datetime
import pickle

In [3]:
embeddings_arcface = []
embeddings_facenet  = []
embeddings_vggface2 = []
database_dir = "Dutta"
for filename in os.listdir(database_dir):
    if filename.lower().endswith(('.png','.jpg',',jpeg')):
        label = os.path.splitext(filename)[0] # to extract the name of the filename
        img_path = os.path.join(database_dir,filename)
        img = cv2.imread(img_path)

        #Detect the faces 
        faces = RetinaFace.detect_faces(img)
        if faces :
            face = faces[next(iter(faces))]
            x1,y1,x2,y2 = face['facial_area']
            face_img = img[y1:y2, x1:x2]

            #Extract the face embeddings from the image 
            arcface_embedding= DeepFace.represent(face_img, model_name='ArcFace', detector_backend="retinaface",enforce_detection= False,align = True)[0]['embedding']
            

            # Convert embedding to float32 and ensure it's a 1D array
            embedding = np.array(embedding).astype('float32').flatten()

            arc_face_embeddings.append(embedding)
            #label = np.array(label)
            #labels.append(label)
            print(f"{label} has been registered")

Dutta_1 has been registered
Dutta_10 has been registered
Dutta_11 has been registered
Dutta_12 has been registered
Dutta_13 has been registered
Dutta_2 has been registered
Dutta_3 has been registered
Dutta_4 has been registered
Dutta_5 has been registered
Dutta_6 has been registered
Dutta_7 has been registered
Dutta_8 has been registered
Dutta_9 has been registered


## Calculate the average value of the embeddings

In [4]:
import cv2
import os
import numpy as np
from scipy.spatial.distance import cosine
from retinaface import RetinaFace
from deepface import DeepFace

# Assuming you have the embeddings and labels stored for each model
embeddings_arcface = np.array(embeddings_arcface)
embeddings_facenet = np.array(embeddings_facenet)
embeddings_vggface2 = np.array(embeddings_vggface2)
labels = np.array(labels)

def predict_label(query_embedding, embeddings_list, labels, weights=None):
    """
    Predict the label of a given query embedding using an ensemble of face recognition models.
    
    Args:
        query_embedding (np.ndarray): The embedding of the query face.
        embeddings_list (list): A list of embedding arrays, one for each face recognition model.
        labels (np.ndarray): The labels corresponding to the known faces.
        weights (list, optional): A list of weights for each model. If not provided, equal weights are used.
        
    Returns:
        str: The predicted label for the query face.
    """
    if weights is None:
        weights = [1 / len(embeddings_list)] * len(embeddings_list)

    distances = []
    for embeddings in embeddings_list:
        distances.append([cosine(query_embedding, emb) for emb in embeddings])

    weighted_distances = np.average(np.array(distances), axis=0, weights=weights)
    nearest_index = np.argmin(weighted_distances)
    return labels[nearest_index]

def predict_label_single(query_embedding, embeddings, labels):
    """
    Predict the label of a given query embedding using distance-based matching.
    
    Args:
        query_embedding (np.ndarray): The embedding of the query face.
        embeddings (np.ndarray): The embeddings of the known faces.
        labels (np.ndarray): The labels corresponding to the known faces.
        
    Returns:
        str: The predicted label for the query face.
    """
    distances = [cosine(query_embedding, emb) for emb in embeddings]
    nearest_index = np.argmin(distances)
    return labels[nearest_index]

# Example usage
query_image_path = "path/to/query/image.jpg"
query_img = cv2.imread(query_image_path)
query_faces = RetinaFace.detect_faces(query_img)

if query_faces:
    for _, face in query_faces.items():
        x1, y1, x2, y2 = face['facial_area']
        query_face_img = query_img[y1:y2, x1:x2]

        query_embedding_arcface = DeepFace.represent(query_face_img, model_name='ArcFace', detector_backend="retinaface", enforce_detection=False, align=True)[0]['embedding']
        query_embedding_facenet = DeepFace.represent(query_face_img, model_name='Facenet', detector_backend="retinaface", enforce_detection=False, align=True)[0]['embedding']
        query_embedding_vggface2 = DeepFace.represent(query_face_img, model_name='VGGFace2', detector_backend="retinaface", enforce_detection=False, align=True)[0]['embedding']

        query_embeddings = [
            np.array(query_embedding_arcface).astype('float32').flatten(),
            np.array(query_embedding_facenet).astype('float32').flatten(),
            np.array(query_embedding_vggface2).astype('float32').flatten()
        ]

        predicted_label = predict_label(query_embeddings, [embeddings_arcface, embeddings_facenet, embeddings_vggface2], labels, weights=[0.5, 0.3, 0.2])
        print(f"Predicted label: {predicted_label}")

        # You can also use the single model prediction approach
        # predicted_label_single = predict_label_single(query_embedding_arcface, embeddings_arcface, labels)
        # print(f"Predicted label (single model): {predicted_label_single}")
else:
    print("No faces detected in the query image.")

ModuleNotFoundError: No module named 'scipy'